In [1]:
import matplotlib
import mxnet as mx
import numpy as np
from mxnet import gluon, nd
from mxnet.gluon.data.vision import transforms
from gluoncv.data import imagenet
from collections import namedtuple
import multiprocessing
from mxnet.contrib.onnx.onnx2mx.import_model import import_model

In [2]:
# Determine and set context
if len(mx.test_utils.list_gpus())==0:
    ctx = [mx.cpu()]
else:
    ctx = [mx.gpu(0)]

# path to imagenet dataset folder
data_dir = '/home/jiang784/Benchmark/dataset/imagenet/val'

# batch size (set to 1 for cpu)
batch_size = 128

# number of preprocessing workers
num_workers = multiprocessing.cpu_count()

# path to ONNX model file
model_path = '/home/jiang784/Benchmark/DL_Benchmark/models/onnx/squeezenet1.0-9.onnx'

In [3]:
sym, arg_params, aux_params = import_model(model_path)


In [4]:
# Define evaluation metrics
acc_top1 = mx.metric.Accuracy()
acc_top5 = mx.metric.TopKAccuracy(5)

In [5]:
# Define image transforms
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

# Load and process input
val_data = gluon.data.DataLoader(
    imagenet.classification.ImageNet(data_dir, train=False).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [6]:
# Load module
mod = mx.mod.Module(symbol=sym, context=ctx, label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

ValueError: [91mYou created Module with Module(..., data_names=['data']) but input with name 'data' is not found in symbol.list_arguments(). Did you mean one of:
	data_0
	conv1_w_0
	conv1_b_0
	fire2/squeeze1x1_w_0
	fire2/squeeze1x1_b_0
	fire2/expand1x1_w_0
	fire2/expand1x1_b_0
	fire2/expand3x3_w_0
	fire2/expand3x3_b_0
	fire3/squeeze1x1_w_0
	fire3/squeeze1x1_b_0
	fire3/expand1x1_w_0
	fire3/expand1x1_b_0
	fire3/expand3x3_w_0
	fire3/expand3x3_b_0
	fire4/squeeze1x1_w_0
	fire4/squeeze1x1_b_0
	fire4/expand1x1_w_0
	fire4/expand1x1_b_0
	fire4/expand3x3_w_0
	fire4/expand3x3_b_0
	fire5/squeeze1x1_w_0
	fire5/squeeze1x1_b_0
	fire5/expand1x1_w_0
	fire5/expand1x1_b_0
	fire5/expand3x3_w_0
	fire5/expand3x3_b_0
	fire6/squeeze1x1_w_0
	fire6/squeeze1x1_b_0
	fire6/expand1x1_w_0
	fire6/expand1x1_b_0
	fire6/expand3x3_w_0
	fire6/expand3x3_b_0
	fire7/squeeze1x1_w_0
	fire7/squeeze1x1_b_0
	fire7/expand1x1_w_0
	fire7/expand1x1_b_0
	fire7/expand3x3_w_0
	fire7/expand3x3_b_0
	fire8/squeeze1x1_w_0
	fire8/squeeze1x1

In [ ]:
# Compute evaluations
Batch = namedtuple('Batch', ['data'])
acc_top1.reset()
acc_top5.reset()
num_batches = int(50000/batch_size)
print('[0 / %d] batches done'%(num_batches))
# Loop over batches
for i, batch in enumerate(val_data):
    # Load batch
    data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
    label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
    # Perform forward pass
    mod.forward(Batch([data[0]]))
    outputs=mod.get_outputs()
    # Update accuracy metrics
    acc_top1.update(label, outputs)
    acc_top5.update(label, outputs)
    if (i+1)%50==0:
        print('[%d / %d] batches done'%(i+1,num_batches))

[0 / 390] batches done
[50 / 390] batches done
[100 / 390] batches done
[150 / 390] batches done
[200 / 390] batches done
[250 / 390] batches done
[300 / 390] batches done
[350 / 390] batches done


In [ ]:
# Print results
_, top1 = acc_top1.get()
_, top5 = acc_top5.get()
print("Top-1 acSqcuracy: {}, Top-5 accuracy: {}".format(top1, top5))

Top-1 accuracy: 0.55994, Top-5 accuracy: 0.7894
